In [ ]:
!odsc conda install -s fspyspark32_p38_cpu_v3

In [ ]:
!odsc data-catalog config --authentication resource_principal --metastore <metastore_id>

Oracle Data Science service sample notebook.

Copyright (c) 2022 Oracle, Inc. All rights reserved. Licensed under the [Universal Permissive License v 1.0](https://oss.oracle.com/licenses/upl).

***

# <font color="red">Feature Engineering for medical data in Feature store </font>
<p style="margin-left:10%; margin-right:10%;">by the <font color="teal">Oracle Cloud Infrastructure Data Science Service.</font></p>

---
# Overview:
---
Managing many datasets, data-sources and transformations for machine learning is complex and costly. Poorly cleaned data, data issues, bugs in transformations, data drift and training serving skew all leads to increased model development time and worse model performance. Here, feature store is well positioned to solve many of the problems since it provides a centralised way to transform and access data for training and serving time and helps defines a standardised pipeline for ingestion of data and querying of data. This notebook demonstrates how to use feature store within a long lasting [Oracle Cloud Infrastructure Data Flow](https://docs.oracle.com/en-us/iaas/data-flow/using/home.htm) cluster.

Compatible conda pack: [PySpark 3.2 and Feature store](https://docs.oracle.com/iaas/data-science/using/conda-pyspark-fam.htm) for CPU on Python 3.8

<div>
    <img src="https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/_images/overview-roles.png"  />
</div>

## Contents:

- <a href="#concepts">1. Introduction</a>
- <a href='#pre-requisites'>1. Pre-requisites</a>
    - <a href='#policies'>2.1 Policies</a>
    - <a href='#prerequisites_authentication'>2.2 Authentication</a>
    - <a href='#prerequisites_variables'>2.3 Variables</a>
- <a href='#featurestore_querying'>3. Feature store querying</a>
    - <a href='#data_exploration'>3.1. Exploration of data in feature store</a>
    - <a href='#load_featuregroup'>3.2. Load feature groups</a>
    - <a href='#explore_featuregroup'>3.3. Explore feature groups</a>
    - <a href='#select_subset_featuregroup'>3.4. Select subset of features</a>
    - <a href='#filter_featuregroup'>3.5. Filter feature groups</a>
    - <a href='#join_featuregroup'>3.6. Apply joins on feature group</a>
    - <a href='#create_dataset'>3.7. Create dataset from multiple or one feature group</a>
    - <a href='#sql_query'>3.8 Free form sql query</a>
- <a href='#ref'>4. References</a>

---

**Important:**

Placeholder text for required values are surrounded by angle brackets that must be removed when adding the indicated content. For example, when adding a database name to `database_name = "<database_name>"` would become `database_name = "production"`.

---

<a id="concepts"></a>
# 1. Introduction

Oracle feature store is a stack based solution that is deployed in the customer enclave using OCI resource manager. Customer can stand up the service with infrastructure in their own tenancy. The service consists of API which are deployed in customer tenancy using resource manager.

The following are some key terms that will help you understand OCI Data Science Feature Store:


* **Feature Vector**: Set of feature values for any one primary/identifier key. Eg. All/subset of features of customer id ‘2536’ can be called as one feature vector.

* **Feature**: A feature is an individual measurable property or characteristic of a phenomenon being observed.

* **Entity**: An entity is a group of semantically related features. The first step a consumer of features would typically do when accessing the feature store service is to list the entities and the entities associated features. Another way to look at it is that an entity is an object or concept that is described by its features. Examples of entities could be customer, product, transaction, review, image, document, etc.

* **Feature Group**: A feature group in a feature store is a collection of related features that are often used together in ml models. It serves as an organizational unit within the feature store for users to manage, version and share features across different ml projects. By organizing features into groups, data scientists and ml engineers can efficiently discover, reuse and collaborate on features reducing the redundant work and ensuring consistency in feature engineering.

* **Feature Group Job**: Feature group job is the execution instance of a feature group. Each feature group job will include validation results and statistics results.

* **Dataset**: A dataset is a collection of feature that are used together to either train a model or perform model inference.

* **Dataset Job**: Dataset job is the execution instance of a dataset. Each dataset job will include validation results and statistics results.

<a id='pre-requisites'></a>
# 2. Pre-requisites

Data Flow Sessions are accessible through the following conda environment:

* **PySpark 3.2, Feature store 1.0 and Data Flow 1.0 (fs_pyspark32_p38_cpu_v1)**

The [Data Catalog Hive Metastore](https://docs.oracle.com/en-us/iaas/data-catalog/using/metastore.htm) provides schema definitions for objects in structured and unstructured data assets. The Metastore is the central metadata repository to understand tables backed by files on object storage. You can customize `fs_pyspark32_p38_cpu_v1`, publish it, and use it as a runtime environment for a Data Flow session cluster. The metastore id of hive metastore is tied to feature store construct of feature store service.


<a id='setup_spark-defaults'></a>
### `spark-defaults.conf`

The `spark-defaults.conf` file is used to define the properties that are used by Spark. A templated version is installed when you install a Data Science conda environment that supports PySpark. However, you must update the template so that the Data Catalog metastore can be accessed. You can do this manually. However, the `odsc data-catalog config` commandline tool is ideal for setting up the file because it gathers information about your environment, and uses that to build the file.

The `odsc data-catalog config` command line tool needs the `--metastore` option to define the Data Catalog metastore OCID. No other command line option is needed because settings have default values, or they take values from your notebook session environment. Following are common parameters that you may need to override.

The `--authentication` option sets the authentication mode. It supports resource principal and API keys. The preferred method for authentication is resource principal, which is sent with `--authentication resource_principal`. If you want to use API keys, then use the `--authentication api_key` option. If the `--authentication` isn't specified, API keys are used. When API keys are used, information from the OCI configuration file is used to create the `spark-defaults.conf` file.

Object Storage and Data Catalog are regional services. By default, the region is set to the region your notebook session is running in. This information is taken from the environment variable, `NB_REGION`. Use the `--region` option to override this behavior.

The default location of the `spark-defaults.conf` file is `/home/datascience/spark_conf_dir` as defined in the `SPARK_CONF_DIR` environment variable. Use the `--output` option to define the directory where to write the file.

You need to determine what settings are appropriate for your configuration. However, the following works for most configurations and is run in a terminal window.

```bash
odsc data-catalog config --authentication resource_principal --metastore <metastore_id>
```
For more assistance, use the following command in a terminal window:

```bash
odsc data-catalog config --help
```

<a id='setup_session'></a>
### Session Setup

The notebook makes connections to the Data Catalog metastore and Object Storage. In the next cell, specify the bucket URI to act as the data warehouse. Use the `warehouse_uri` variable with the `oci://<bucket_name>@<namespace_name>/<key>` format. Update the variable `metastore_id` with the OCID of the Data Catalog metastore.

<a id='policies'></a>
### 2.1. Policies
This section covers the creation of dynamic groups and policies needed to use the service.

* [Data Flow Policies](https://docs.oracle.com/iaas/data-flow/using/policies.htm/)
* [Data Catalog Metastore Required Policies](https://docs.oracle.com/en-us/iaas/data-catalog/using/metastore.htm)
* [Getting Started with Data Flow](https://docs.oracle.com/iaas/data-flow/using/dfs_getting_started.htm)
* [About Data Science Policies](https://docs.oracle.com/iaas/data-science/using/policies.htm)

<a id="prerequisites_authentication"></a>
### 2.2. Authentication
The [Oracle Accelerated Data Science SDK (ADS)](https://docs.oracle.com/iaas/tools/ads-sdk/latest/index.html) controls the authentication mechanism with the notebook cluster.<br>
To setup authentication use the ```ads.set_auth("resource_principal")``` or ```ads.set_auth("api_key")```.

In [ ]:
import ads
ads.set_auth(auth="resource_principal", client_kwargs={"fs_service_endpoint": "{api_endpoint}/20230101"})

<a id="prerequisites_variables"></a>
### 2.3. Variables
To run this notebook, you must provide some information about your tenancy configuration. To create and run a feature store, you must specify a `<compartment_id>` and bucket `<metastore_id>` for offline feature store.

In [ ]:
import os

compartment_id = os.environ.get("NB_SESSION_COMPARTMENT_OCID")
metastore_id = "<metastore_id>"

<a id="featurestore_querying"></a>
# 3. Feature group querying
By default the **PySpark 3.2, Feature store and Data Flow** conda environment includes pre-installed [great-expectations](https://legacy.docs.greatexpectations.io/en/latest/reference/core_concepts/validation.html) and OCI ML monitoring libraries. The joining functionality is heavily inspired by the APIs used by Pandas to merge, join or filter DataFrames. The APIs allow you to specify which features to select from which feature group, how to join them and which features to use in join conditions.



In [ ]:
import pandas as pd
from ads.feature_store.feature_store import FeatureStore
from ads.feature_store.feature_group import FeatureGroup
from ads.feature_store.model_details import ModelDetails
from ads.feature_store.dataset import Dataset
from ads.feature_store.common.enums import DatasetIngestionMode

from ads.feature_store.feature_group_expectation import ExpectationType
from great_expectations.core import ExpectationSuite, ExpectationConfiguration
from ads.feature_store.feature_store_registrar import FeatureStoreRegistrar

<a id="data_exploration"></a>
### 3.1. Exploration of data in feature store

The dataset contains Electronic Health Record predictions collected from a private Hospital in Indonesia. It contains the patient's laboratory test results used to determine the next patient's treatment whether in care or out care.


In [ ]:
patient_result_df = pd.read_csv("https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/data/EHR/data-ori.csv")
patient_result_df.head()

#### Dataset Overview 

In [ ]:
print(f"The dataset contains {patient_result_df.shape[0]} rows and {patient_result_df.shape[1]} columns")

# get all the features
features = [feat for feat in patient_result_df.columns if feat !='SOURCE']
num_features = [feat for feat in features if patient_result_df[feat].dtype != object]
cat_features = [feat for feat in features if patient_result_df[feat].dtype == object]

print(f"Total number of features : {len(features)}")
print(f"Number of numerical features : {len(num_features)}")
print(f"Number of categorical features : {len(cat_features)}\n")

<b> Missing Values</b>

In [ ]:
patient_result_df.isna().mean().to_frame(name='Missing %')

<b>Unique Values</b>

In [ ]:
patient_result_df.nunique().to_frame(name='# of unique values')

<a id="load_featuregroup"></a>
### 3.2. Create feature store logical entities

#### 3.2.1 Feature Store
Feature store is the top level entity for feature store service

In [ ]:
feature_store_resource = (
    FeatureStore().
    with_description("Electronic Heath Data consisting of Patient Test Results").
    with_compartment_id(compartment_id).
    with_name("EHR details").
    with_offline_config(metastore_id=metastore_id)
)

<a id="create_feature_store"></a>
##### Create Feature Store

Call the ```.create()``` method of the Feature store instance to create a feature store.

In [ ]:
feature_store = feature_store_resource.create()
feature_store

#### 3.2.2 Entity
An entity is a group of semantically related features.

In [ ]:
entity = feature_store.create_entity(
    name="EHR",
    description="Electronic Health Record predictions"
)
entity

#### 3.2.3 Transformation
Transformations in a feature store refers to the operations and processes applied to raw data to create, modify or derive new features that can be used as inputs for ML Models


Below python function defines 2 PANDAS transformations which are used for feature engineering.<code>label_encoder_transformation</code> performs label encoding to convert  categorical variables into numerical format. It take the column label as input keyword argument that needs to be encoded. Another transformation is  <code>min_max_scaler</code> which scale the features to a range of 0 to 1<br>
<code>chained_transformation</code> chains both of these transformation by first performing imputation and then
doing label encoding on the returned dataframe.

In [ ]:
def chained_transformation(patient_result_df, **transformation_args):
    def label_encoder_transformation(patient_result_df, **transformation_args):
        from sklearn.preprocessing import LabelEncoder
        # creating instance of labelencoder
        labelencoder = LabelEncoder()
        result_df = patient_result_df.copy()
        column_labels= transformation_args.get("label_encode_column")
        if isinstance(column_labels,list):
            for col in column_labels:
                result_df[col] = labelencoder.fit_transform(result_df[col])
        elif isinstance(column_labels, str):
             result_df[column_labels] = labelencoder.fit_transform(result_df[column_labels])
        else:
            return None
        return result_df
    
    def min_max_scaler(patient_result_df, **transformation_args):
        from sklearn.preprocessing import MinMaxScaler
        final_result_df = patient_result_df.copy()
        scaler = MinMaxScaler(feature_range=(0, 1))
        column_labels= transformation_args.get("scaling_column_labels")
        final_result_df[column_labels] = scaler.fit_transform(final_result_df[column_labels])
        return patient_result_df
    
    def feature_removal(input_df, **transformation_args):
        output_df = input_df.copy()
        output_df.drop(transformation_args.get("redundant_feature_label"), axis=1, inplace=True)
        return output_df

    out1 = label_encoder_transformation(patient_result_df, **transformation_args)
    out2 = min_max_scaler(out1, **transformation_args)
    return feature_removal(out2, **transformation_args)

In [ ]:
transformation_args = {
    "label_encode_column": ["SEX","SOURCE"],
    "scaling_column_labels": num_features,
    "redundant_feature_label": ["MCH", "MCHC", "MCV"]
}


In [ ]:
from ads.feature_store.transformation import Transformation,TransformationMode

transformation = (
    Transformation()
    .with_name("chained_transformation")
    .with_feature_store_id(feature_store.id)
    .with_source_code_function(chained_transformation)
    .with_transformation_mode(TransformationMode.PANDAS)
    .with_description("transformation to perform feature engineering")
    .with_compartment_id(compartment_id)
)

transformation.create()

#### 3.2.3 Feature group
A feature group is an object that represents a logical group of time-series feature data as it is found in a datasource.

<a id="create_feature_group_ehr"></a>
##### Patient's EHR Feature Group

Create feature group for EHR

<div>
    <img src="https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/_images/feature_group_flights.gif" width="700" height="350" />
</div>

In [ ]:
feature_group_ehr = (
    FeatureGroup()
    .with_feature_store_id(feature_store.id)
    .with_primary_keys([])
    .with_name("ehr_feature_group")
    .with_entity_id(entity.id)
    .with_compartment_id(compartment_id)
    .with_schema_details_from_dataframe(patient_result_df)
    .with_transformation_id(transformation.id)
    .with_transformation_kwargs(transformation_args)
)

In [ ]:
feature_group_ehr.create()

In [ ]:
feature_group_ehr.show()

##### Expectations

Update feature group with expectations

In [ ]:
expectation_suite_ehr = ExpectationSuite(
    expectation_suite_name="test_hcm_df"
)
expectation_suite_ehr.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={"column": "AGE"},
    )
)
expectation_suite_ehr.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "HAEMOGLOBINS", "min_value": 0, "max_value": 30},
    )
)

In [ ]:
from ads.feature_store.common.enums import ExpectationType

feature_group_ehr.with_expectation_suite(expectation_suite_ehr, expectation_type = ExpectationType.STRICT)
feature_group_ehr.update()

In [ ]:
feature_group_ehr.materialise(patient_result_df)

<a id="explore_featuregroup"></a>
### 3.3. Explore feature groups

In [ ]:
feature_group_ehr.get_features_df()

In [ ]:
feature_group_ehr.select().show()

In [ ]:
feature_group_ehr.get_statistics().to_pandas()

In [ ]:
feature_group_ehr.get_validation_output().to_pandas()

<a id="select_subset_featuregroup"></a>
### 3.4. Select subset of features

In [ ]:
feature_group_ehr.select(['AGE', 'SEX', 'SOURCE']).show()

<a id="filter_featuregroup"></a>
### 3.5. Filter feature groups

In [ ]:
feature_group_ehr.filter(feature_group_ehr.SOURCE == "1").show()

<a id="create_dataset"></a>
### 3.7 Create dataset
A dataset is a collection of feature snapshots that are joined together to either train a model or perform model inference.

<div>
    <img src="https://objectstorage.us-ashburn-1.oraclecloud.com/p/hh2NOgFJbVSg4amcLM3G3hkTuHyBD-8aE_iCsuZKEvIav1Wlld-3zfCawG4ycQGN/n/ociodscdev/b/oci-feature-store/o/beta/_images/fs_ehr_dataset.gif" width="700" height="350" />
</div>

In [ ]:
query = feature_group_ehr.select() 
query.show()

In [ ]:
dataset = (
    Dataset()
    .with_description("Combined dataset for EHR")
    .with_compartment_id(compartment_id)
    .with_name("ehr_dataset")
    .with_entity_id(entity.id)
    .with_feature_store_id(feature_store.id)
    .with_query(query.to_string())
)

<a id="create_dataset"></a>
##### Create Dataset

Call the ```.create()``` method of the Dataset instance to create a dataset.

In [ ]:
dataset.create()

In [ ]:
dataset.materialise()

### Interoperability with model

In [ ]:
model_details = ModelDetails().with_items(["ocid1.datasciencemodel..."])
dataset.add_models(model_details)

<a id="visualise_lineage"></a>
##### Visualise lineage

Use the ```.show()``` method on the Dataset instance to visualize the lineage of the dataset.

In [ ]:
dataset.show()

In [ ]:
dataset.profile().show()

In [ ]:
dataset.preview().show()

<a id='ref'></a>
# References

- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)